In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import re
import os
import shutil
from tqdm.autonotebook import tqdm, trange

from IPython.display import Image, display

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

<ipython-input-1-daa88f1124fc>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [179]:
classes = pd.read_csv('classes.csv')
classes.drop('Unnamed: 0', axis=1, inplace=True)
data = pd.read_csv('train.csv')
# data.drop('Unnamed: 0', axis=1, inplace=True)

In [23]:
data.head()

,class_id,image
0,1221,3kVLfN1tTupzPhFgWY0yeOnqoD6RwG57HlSMvBK4.jpg
1,1209,p5FeqKS7BHknQM4OvbzZs60giLthaCW83VGDx1Y9.jpg
2,865,3GHsrhop0FJ58EBjLDgQed2NvI4UK1ybZRAuTl6n.jpg
3,1222,DqoLbEG8A9lJCaR7ucOdiP05rXxyjg4WwMSh2VFs.jpg
4,542,TV1zBo5gLt34i76pAFya90HuJWNjkZSCwbKDevnO.jpg


In [47]:
classes['id']

0          0
1          1
2          2
3          3
4          4
        ... 
4391    4391
4392    4392
4393    4393
4394    4394
4395    4395
Name: id, Length: 4396, dtype: int64

In [3]:
data = data.sort_values(by=['class_id'])

In [48]:
exist_class = []
for class_id in classes.id:
    for id in data.class_id:
        if class_id == id and class_id not in exist_class:
            exist_class.append(class_id)

In [50]:
len(exist_class)

4325

In [70]:
for id in data.class_id:
    if(not os.path.exists(f"images/images/{data[data['class_id'] == id].image.values[0]}")):
        print(f"{data[data['class_id'] == id].class_id.values[0]}")

## Убираю ненужные классы

In [180]:
class_distribution = classes.groupby('brand', as_index=False).count().drop('model', axis=1)
class_distribution.rename({'id': 'count'}, axis='columns', inplace=True)
class_distribution = class_distribution.sort_values(by=['count'])
brands = class_distribution[class_distribution['count'] > 4].brand
brands = brands.tolist()

In [181]:
brands

['unbranded',
 'yeezy',
 'onitsuka tiger',
 'jordan',
 'vans',
 'new balance',
 'puma',
 'adidas',
 'nike',
 'asics',
 'reebok',
 'mizuno']

In [182]:
brands = [r'unbranded*',
 r'yeezy*',
 r'onitsuka tiger*',
 r'jordan*',
 r'vans*',
 r'new balance*',
 r'puma*',
 r'adidas*',
 r'nike*',
 r'asics*',
 r'reebok*',
 r'mizuno*']

In [183]:
classes['model'] = classes['model'].replace(brands,['','','','','','','','','','','',''],regex=True)
classes['model'] = classes['model'].apply(lambda x: re.sub(r'[^\w\s]', '', x).strip())
classes['model'] = classes['model'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [198]:
class_model = classes.groupby(['brand', 'model'])['id'].apply(lambda x: ';'.join(map(str, x))).reset_index()
class_model[class_model['brand'] == 'yeezy']

,brand,model,id
3939,yeezy,500,2744
3940,yeezy,boost 350 v2,730;4099
3941,yeezy,foam rnnr,769
3942,yeezy,slide pure,706


In [94]:
class_model[class_model['model'] == '']

,brand,model,id
46,new balance,,46
450,puma,,450
740,adidas,,740
813,vans,,813
839,rick owens,,839
1218,reebok,,1218
1516,mizuno,,1516
1916,asics,,1916
2930,jordan,,2930
4061,nike,,4061


In [101]:
class_model.id

0          0
1          1
2          2
3          3
5          5
        ... 
4391    4391
4392    4392
4393    4393
4394    4394
4395    4395
Name: id, Length: 3943, dtype: int64

In [200]:
len(classes_dict.keys())

3881

In [206]:
sss = 0
for x in class_model.id:
    if ';' in str(x):
        sss+=len(x.split(';'))
print(sss)

876


In [178]:
len(ones_class)

252

## Разделение фоток по папкам их класса для train

In [199]:
from collections import defaultdict
classes_dict = defaultdict(list)

for index, row in data.iterrows():
    image_name = row['image']
    if row['class_id'] in class_model.id:
        classes_dict[row['class_id']].append(image_name)

In [103]:
ones_class = []
for key in classes_dict:
    if len(classes_dict[key]) < 2:
        ones_class.append(key)

In [104]:

# Путь к главной папке, содержащей все изображения
main_folder_path = 'images/images/'

# Создать папки для каждого класса, если они не существуют
for class_name in data['class_id'].unique():
    if class_name not in ones_class:
        class_folder_path = os.path.join(main_folder_path, str(class_name))
        if not os.path.exists(class_folder_path):
            os.makedirs(class_folder_path)

# for class_name in classes_dict.keys():
#     if class_name not in ones_class:
#         class_folder_path = os.path.join(main_folder_path, str(class_name))
#         if not os.path.exists(class_folder_path):
#             os.makedirs(class_folder_path)

SIZE_DELETE = 1000
# Перенести каждую картинку из главной папки в папку класса, если она существует в DataFrame pandas
for index, row in tqdm(data.iterrows(),total=data.shape[0]):
    image_name = row['image']
    class_name = row['class_id']
    source_path = os.path.join(main_folder_path, image_name)
    if os.path.getsize(source_path) < SIZE_DELETE:
        os.remove(source_path)
    elif class_name not in ones_class:
        dest_folder_path = os.path.join(main_folder_path, str(class_name))
        dest_path = os.path.join(dest_folder_path, image_name)
        if os.path.exists(source_path):
            shutil.move(source_path, dest_path)


In [108]:
from pathlib import Path

counter_in_folder = []
for index, row in tqdm(data.iterrows(),total=data.shape[0]):
    folder = Path(f'./images/images/{row["class_id"]}')

    if not folder.is_dir():
#         raise ValueError(f"[{folder}] не существует или не является директорией")
        pass
    else:
        folder_counts = sum(1 for x in folder.iterdir())
        counter_in_folder.append([folder_counts, row['class_id']])

In [170]:
sumAbove = []
sumMore = []
for counter in counter_in_folder:
    if counter[0] < 5 and counter[1] not in sumAbove:
        sumAbove.append(counter[1])
    elif counter[0] >= 30 and counter[1] not in sumMore:
        sumMore.append(counter[1])
print(len(sumAbove), len(sumMore))

1594 263


## Разделение трейна на train и val

In [ ]:
!pip install split-folders 

In [171]:
from pathlib import Path

for index, row in tqdm(data.iterrows(),total=data.shape[0]):
    folder = Path(f'./images/train/{row["class_id"]}')

    if not folder.is_dir():
#         raise ValueError(f"[{folder}] не существует или не является директорией")
        pass
    else:
        folder_counts = sum(1 for x in folder.iterdir())
        if(folder_counts < 2):
            shutil.rmtree(folder)

In [22]:
import splitfolders

input_folder = "./images/train"
splitfolders.ratio(input_folder, 'train_test', ratio = (0.8, 0.2), seed=42, group_prefix=None)

Copying files: 47284 files [06:15, 126.08 files/s]


# чет другое

In [7]:
merged = pd.read_csv('merged.csv')
merged = merged.drop(['Unnamed: 0','Unnamed: 0_x','class_id','Unnamed: 0_y','id','model'],axis=1)
merged.head()

,image,right_id
0,3kVLfN1tTupzPhFgWY0yeOnqoD6RwG57HlSMvBK4.jpg,1091
1,lKUEs3gWjhLm28vFCo6ruacX0JZyGkMNBwTHVbOR.jpg,1091
2,ruAzlwKWCjsiVtLY27fmkQUeEnGODb6RoMhpqPN8.jpg,1091
3,294rLQNFBShbpOIcHx5mwgu1RzyUnavW783D0KfT.jpg,1091
4,vYZzN9cmAg8wMuf5TxPUdGeOinWIDX6L7pSRh0J2.jpg,1091


In [4]:
data = pd.read_csv('cleaned_data.csv')
data.head()

,Unnamed: 0,model,id
0,0,new balance 574,4
1,1,new balance 327,5
2,2,new balance 990v4,21
3,3,new balance 237,24
4,4,new balance 373,25


In [6]:
models = data['model']
models

0            new balance 574
1            new balance 327
2          new balance 990v4
3            new balance 237
4            new balance 373
                ...         
3840               puma rs2k
3841       puma reboundlayup
3842    puma muensterclassic
3843            puma dayzero
3844       puma strunnerv2sd
Name: model, Length: 3845, dtype: object

In [9]:
from sklearn.metrics.pairwise import cosine_distances as cos_dis

def levenstein(str_1, str_2):
    n, m = len(str_1), len(str_2)
    if n > m:
        str_1, str_2 = str_2, str_1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if str_1[j - 1] != str_2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

0


In [13]:
models_lev = []
for i in range(1,len(models)):
    for j in range(i+1, len(models)):
        models_lev.append([models[i],models[j],levenstein(models[i],models[j])])

In [15]:
models_lev_1 = []
for l in models_lev:
    if l[2] <= 1:
        models_lev_1.append(l)

In [19]:
df_lev = pd.DataFrame(models_lev_1, columns=['model1','model2','levenstein'])
df_lev.head()

,model1,model2,levenstein
0,new balance 327,new balance 927,1
1,new balance 327,new balance 527,1
2,new balance 990v4,new balance 990v5,1
3,new balance 990v4,new balance 940v4,1
4,new balance 990v4,new balance 990v6,1


In [20]:
df_lev.to_csv('models_levenstein.csv', index=False)

In [10]:

# Путь к главной папке, содержащей все изображения
main_folder_path = 'images/'

# Создать папки для каждого класса, если они не существуют
for class_name in merged['right_id'].unique():
    class_folder_path = os.path.join(main_folder_path, str(class_name))
    if not os.path.exists(class_folder_path):
        os.makedirs(class_folder_path)

SIZE_DELETE = 1000
# Перенести каждую картинку из главной папки в папку класса, если она существует в DataFrame pandas
for index, row in tqdm(merged.iterrows(),total=merged.shape[0]):
    image_name = row['image']
    class_name = row['right_id']
    source_path = os.path.join(main_folder_path, image_name)
    if os.path.getsize(source_path) < SIZE_DELETE:
        os.remove(source_path)
    else:
        dest_folder_path = os.path.join(main_folder_path, str(class_name))
        dest_path = os.path.join(dest_folder_path, image_name)
        if os.path.exists(source_path):
            shutil.move(source_path, dest_path)


# возвращаем айдишки

In [2]:
merged = pd.read_csv('merged.csv')
merged = merged.drop(['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y'], axis = 1)
merged.head()

,class_id,image,model,id,right_id
0,1221,3kVLfN1tTupzPhFgWY0yeOnqoD6RwG57HlSMvBK4.jpg,reebok reebokclassic,1221,1091
1,1221,lKUEs3gWjhLm28vFCo6ruacX0JZyGkMNBwTHVbOR.jpg,reebok reebokclassic,1221,1091
2,1221,ruAzlwKWCjsiVtLY27fmkQUeEnGODb6RoMhpqPN8.jpg,reebok reebokclassic,1221,1091
3,1221,294rLQNFBShbpOIcHx5mwgu1RzyUnavW783D0KfT.jpg,reebok reebokclassic,1221,1091
4,1221,vYZzN9cmAg8wMuf5TxPUdGeOinWIDX6L7pSRh0J2.jpg,reebok reebokclassic,1221,1091


In [3]:
ids = merged['id'].unique()
right_ids = merged['right_id'].unique()

In [4]:
ids_rights = dict()
for i in range(len(ids)):
    ids_rights[right_ids[i]] = ids[i]

In [5]:
subm = pd.read_csv('submit (4).csv')
subm.head()

,class_id,image
0,3500,mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg
1,3554,Dvtu5QLrVFq0JxjlcgmK94n3SC68iAphHXoBfzYM.jpg
2,1902,mldYZ9OiXh5P0ERHNcKLp78SbMtfkBx2rwJeszoa.jpg
3,1,tZaCF4XK2hPLTlW8YRdAnMD5mfSxzBujwVpUJ7b0.jpg
4,3339,73gEZNzIV9SLcf14wkrTYUoHJOMuDetsvihP0mAq.jpg


In [6]:
def replace_id(id):
    return ids_rights[id]

In [7]:
new_subm = pd.read_csv('submit (4).csv')

In [8]:
new_subm['class_id'] = subm['class_id'].apply(replace_id)

In [9]:
new_subm

,class_id,image
0,4006,mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg
1,4065,Dvtu5QLrVFq0JxjlcgmK94n3SC68iAphHXoBfzYM.jpg
2,2153,mldYZ9OiXh5P0ERHNcKLp78SbMtfkBx2rwJeszoa.jpg
3,4,tZaCF4XK2hPLTlW8YRdAnMD5mfSxzBujwVpUJ7b0.jpg
4,3814,73gEZNzIV9SLcf14wkrTYUoHJOMuDetsvihP0mAq.jpg
...,...,...
31713,3826,RngzptsIy9bHYMEq32Cx5lDN01uXedfimoTackGP.jpg
31714,3814,nDqGbFxgpfvJ2A43Z5r6VRXywTHaP7MelEBt8zi0.jpg
31715,3814,2RABe1VqCD35sItMiKwcb6kr9olXaxW4PfQ8hNJH.jpg
31716,3814,zcbPK2lH70qAkJfiQZrSDLxe5gYyEB9Uv4jNTMXF.jpg


In [10]:
new_subm.to_csv('submit_new.csv', index=False)